In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv


In [2]:
# Load data
X_train_orig = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
X_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

X_train_orig.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Prepare data
X_train = X_train_orig.copy()

# Convert original label data to categorical, i.e.
# label  ->  0  1  2  3  4  5  6  7  8  9
# ---------------------------------------
#     1  ->  0  1  0  0  0  0  0  0  0  0
#     0  ->  1  0  0  0  0  0  0  0  0  0
#     1  ->  0  1  0  0  0  0  0  0  0  0
#     4  ->  0  0  0  0  1  0  0  0  0  0
#     0  ->  1  0  0  0  0  0  0  0  0  0
def label_to_cat(row):
    row[row.label] = 1
    return row
y_train = pd.DataFrame(np.zeros((len(X_train.index), 10)))
y_train['label'] = X_train.label
y_train = y_train.apply(lambda x: label_to_cat(x), axis=1)
y_train = y_train.drop(columns=['label'])
X_train = X_train.drop(columns=['label'])
y_train.head()

,0,1,2,3,4,5,6,7,8,9
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Craete model
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten

model = Sequential()
model.add(Flatten(input_shape=(28 * 28,)))
model.add(Dense(28 * 28, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Fit model
model.fit(X_train, y_train, epochs=5, batch_size=32)

Train on 42000 samples
Epoch 1/5
42000/42000 [==============================] - 6s 137us/sample - loss: 3.4826 - accuracy: 0.9086
Epoch 2/5
42000/42000 [==============================] - 5s 124us/sample - loss: 0.4509 - accuracy: 0.9417
Epoch 3/5
42000/42000 [==============================] - 5s 123us/sample - loss: 0.3859 - accuracy: 0.9451
Epoch 4/5
42000/42000 [==============================] - 5s 122us/sample - loss: 0.3070 - accuracy: 0.9499
Epoch 5/5
42000/42000 [==============================] - 5s 123us/sample - loss: 0.3229 - accuracy: 0.9488


In [6]:
# Submit predictions
predictions = model.predict(X_test.values)

output = pd.DataFrame({'ImageId': pd.Series(range(1, len(predictions) + 1)),
                       'Label': predictions.argmax(axis=1)})
output.to_csv('submission.csv', index=False)
print("Submission was successfully saved!")

Submission was successfully saved!
